# Segmenting and Clustering Neighborhoods in Toronto - Part 1 
(Web scrapping wiki page)

Import the required libraries and read the wiki page into html_doc variable

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
results = requests.get(wiki_url)
#print(results)
html_doc = results.content
#print(html_doc)
print("complete")

complete


#### Use BeautifulSoup library to identify the class table by name "wikitable sortable" and collect all data into a data array

In [3]:
soup = BeautifulSoup(html_doc, 'html.parser')
#print(soup.prettify())

table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
data = []

for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values
print(data[:5])

[[], ['M1A', 'Not assigned', 'Not assigned'], ['M2A', 'Not assigned', 'Not assigned'], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village']]


#### Clean data to remove Borough's which are Not Assigned and populate details for Neighbourhood which are Not Assigned. Assing the clean data to a new array data_new



In [4]:
data_new = []
data_new.append(['PostCode','Borough','Neighbourhood'])
for i in range(0,len(data)):
    if len(data[i]) == 0:
        continue
    elif data[i][1] == "Not assigned":
        continue
    elif data[i][1] != "Not assigned" and data[i][2] == "Not assigned":
        data[i][2] = data[i][1]
        data_new.append(data[i])   
    else:
        data_new.append(data[i])   
print(data_new[:5])

[['PostCode', 'Borough', 'Neighbourhood'], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Harbourfront'], ['M5A', 'Downtown Toronto', 'Regent Park']]


#### Create Dataframe from the array created earlier and index based on Postcode to find the unique values. Once the unique values are identified, concatenate the Neighbourhood details into a single entitiy and prepare a final dataframe

In [5]:

df = pd.DataFrame(data_new[1:],columns=data_new[0])

df.set_index("PostCode", inplace=True)

indx = list(df.index.unique())

final_arr = np.array([['PostCode','Borough','Neighbourhood']])
for j in range(0,len(indx)):
#for j in range(0,3):

    tmp_arr = list(df.loc[[indx[j]],['Neighbourhood']].values)
    
    tmp_str = ""
    for k in range(0,len(tmp_arr)):
        val = tmp_arr[k].tolist()
        
        if k == 0:
            tmp_str = val[0]
        else:
            tmp_str = tmp_str + "," + val[0]

    borough_details = np.unique(list(df.loc[[indx[j]],['Borough']].values))

    borough_str = borough_details[0]

    arr = [indx[j],borough_str,tmp_str]
    final_arr = np.append(final_arr, [arr], axis = 0) 

final_dataset = pd.DataFrame(final_arr[1:], columns=final_arr[0])
final_dataset.head()


,PostCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [6]:
final_dataset.shape

(103, 3)

# Segmenting and Clustering Neighborhoods in Toronto - Part 2

In [7]:
#!wget -q -O 'toronto_geo_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


#### Load the toronto geo data from csv and set index on PostCode column in the required dataframes

In [8]:
geo_data = pd.read_csv('toronto_geo_data.csv')

geo_data = geo_data.rename(columns={'Postal Code': 'PostCode'})
geo_data.set_index('PostCode',inplace=True)
geo_data.head()


,Latitude,Longitude
PostCode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [9]:
final_dataset.set_index('PostCode',inplace=True)
final_dataset.head()

,Borough,Neighbourhood
PostCode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Queen's Park


In [10]:
df_2 = final_dataset.join(geo_data)
df_2.reset_index(drop=False, inplace=True)
df_2.head()

,PostCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [11]:
df_2.shape

(103, 5)

# Segmenting and Clustering Neighborhoods in Toronto - Part 3

Exploring the neighbourhood of toronto

In [14]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
from pandas.io.json import json_normalize 

#### Use geopy library to get the latitude and longitude values of Toronto.

In [15]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with all neighbourhoods

In [16]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
neighborhoods = df_2

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Create a dataframe of all Boroughs containing Toronto and visualize those co-ordinates on a map of Toronto

In [17]:
toronto_boro_data = neighborhoods[neighborhoods['Borough'].str.lower().str.contains("toronto")].reset_index(drop=True)
toronto_boro_data.head()

,PostCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [18]:
toronto_boro_data.shape

(38, 5)

In [19]:
map_toronto_boro = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(toronto_boro_data['Latitude'], toronto_boro_data['Longitude'], toronto_boro_data['Borough'], toronto_boro_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_boro)  
    
map_toronto_boro

In [20]:
CLIENT_ID = 'KE******4TT1CTU2' # your Foursquare ID
CLIENT_SECRET = '5HQN2A**********HYVQJ3Q' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

# Explore the first neighbourhood of the dataframe.
In this section we will

1. Explore the first neighborhood in our dataframe.
2. Get the neighborhood's latitude and longitude values.
3. Get the top 100 venues that are within a radius of 500 meters.
4. Map the latitudes and longitudes.

In [21]:
neighborhood_latitude = toronto_boro_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_boro_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_boro_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront,Regent Park are 43.6542599, -79.3606359.


In [22]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [23]:
results = requests.get(url).json()
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Roselle Desserts,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",43.653447,-79.362017
1,Tandem Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.653559,-79.361809
2,Cooper Koo YMCA,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",43.653191,-79.357947
3,Morning Glory Cafe,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",43.653947,-79.361149
4,Body Blitz Spa East,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",43.654735,-79.359874


#### Create function to extract the venue categories 

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo YMCA,Gym / Fitness Center,43.653191,-79.357947
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,Body Blitz Spa East,Spa,43.654735,-79.359874


#### Plot a map of the all the venue for the first neighbourhood

In [26]:
map_neighourbood_1 = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=15)
label_1 = folium.Popup(neighborhood_name, parse_html=True)
folium.Marker(location=[neighborhood_latitude, neighborhood_longitude], popup=label_1).add_to(map_neighourbood_1)

for lat, lng, name, categories in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_neighourbood_1)  
    
map_neighourbood_1

# Explore all the neighbourhood of the dataframe.
In this section we will

1. Explore all neighborhood in our dataframe.
2. Get the neighborhood's latitude and longitude values via function.
3. Get the top 100 venues that are within a radius of 500 meters.

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
toronto_boro_data.head()

,PostCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [29]:
toronto_venues = getNearbyVenues(names=toronto_boro_data['Neighbourhood'],
                                   latitudes=toronto_boro_data['Latitude'],
                                   longitudes=toronto_boro_data['Longitude']
                                  )


Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city


In [30]:
print(toronto_venues.shape)
toronto_venues.head()

(1713, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront,Regent Park",43.65426,-79.360636,Cooper Koo YMCA,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [31]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 238 uniques categories.


#### Analyze Each Neighborhood

In [32]:
toronto_venues[['Neighborhood']].head()

,Neighborhood
0,"Harbourfront,Regent Park"
1,"Harbourfront,Regent Park"
2,"Harbourfront,Regent Park"
3,"Harbourfront,Regent Park"
4,"Harbourfront,Regent Park"


In [33]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']])
#toronto_onehot['Neighborhood'] = None
#toronto_onehot.reset_index()
toronto_onehot['Neighborhood'] = toronto_venues[['Neighborhood']]


# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
#toronto_venues['Neighborhood'].head()

toronto_onehot.head()

,Neighborhood,Venue Category_Accessories Store,Venue Category_Adult Boutique,Venue Category_Afghan Restaurant,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,...,Venue Category_Train Station,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Video Game Store,Venue Category_Vietnamese Restaurant,Venue Category_Whisky Bar,Venue Category_Wine Bar,Venue Category_Wine Shop,Venue Category_Wings Joint,Venue Category_Women's Store,Venue Category_Yoga Studio
0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
toronto_onehot.shape

(1713, 239)

#### group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [35]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Venue Category_Accessories Store,Venue Category_Adult Boutique,Venue Category_Afghan Restaurant,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,...,Venue Category_Train Station,Venue Category_Vegetarian / Vegan Restaurant,Venue Category_Video Game Store,Venue Category_Vietnamese Restaurant,Venue Category_Whisky Bar,Venue Category_Wine Bar,Venue Category_Wine Shop,Venue Category_Wings Joint,Venue Category_Women's Store,Venue Category_Yoga Studio
0,"Adelaide,King,Richmond",0.01,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.01,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.000000
1,Berczy Park,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.043478
3,Business reply mail Processing Centre969 Eastern,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.00,0.0,0.0,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,...,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000


In [36]:
toronto_grouped.shape

(38, 239)

In [37]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                                venue  freq
0          Venue Category_Coffee Shop  0.07
1                 Venue Category_Café  0.05
2           Venue Category_Steakhouse  0.04
3  Venue Category_American Restaurant  0.04
4                  Venue Category_Gym  0.04


----Berczy Park----
                               venue  freq
0         Venue Category_Coffee Shop  0.07
1        Venue Category_Cocktail Bar  0.05
2          Venue Category_Restaurant  0.05
3                 Venue Category_Pub  0.04
4  Venue Category_Seafood Restaurant  0.04


----Brockton,Exhibition Place,Parkdale Village----
                                  venue  freq
0            Venue Category_Coffee Shop  0.13
1         Venue Category_Breakfast Spot  0.09
2                   Venue Category_Café  0.09
3  Venue Category_Performing Arts Venue  0.09
4     Venue Category_Italian Restaurant  0.04


----Business reply mail Processing Centre969 Eastern----
                               venue 

#### Putting data into dataframe

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Venue Category_Coffee Shop,Venue Category_Café,Venue Category_Steakhouse,Venue Category_American Restaurant,Venue Category_Thai Restaurant,Venue Category_Gym,Venue Category_Restaurant,Venue Category_Hotel,Venue Category_Clothing Store,Venue Category_Bar
1,Berczy Park,Venue Category_Coffee Shop,Venue Category_Cocktail Bar,Venue Category_Restaurant,Venue Category_Farmers Market,Venue Category_Steakhouse,Venue Category_Bakery,Venue Category_Cheese Shop,Venue Category_Café,Venue Category_Seafood Restaurant,Venue Category_Pub
2,"Brockton,Exhibition Place,Parkdale Village",Venue Category_Coffee Shop,Venue Category_Performing Arts Venue,Venue Category_Café,Venue Category_Breakfast Spot,Venue Category_Yoga Studio,Venue Category_Stadium,Venue Category_Grocery Store,Venue Category_Furniture / Home Store,Venue Category_Italian Restaurant,Venue Category_Falafel Restaurant
3,Business reply mail Processing Centre969 Eastern,Venue Category_Light Rail Station,Venue Category_Garden,Venue Category_Garden Center,Venue Category_Restaurant,Venue Category_Recording Studio,Venue Category_Auto Workshop,Venue Category_Skate Park,Venue Category_Brewery,Venue Category_Fast Food Restaurant,Venue Category_Spa
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,Venue Category_Boutique,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Harbor / Marina,Venue Category_Boat or Ferry,Venue Category_Sculpture Garden


#### Cluster Neighborhoods

In [44]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

toronto_grouped_clustering.shape
#toronto_grouped_clustering.head()


(38, 238)

In [67]:
df_tmp = toronto_venues[['Neighborhood','Neighborhood Latitude','Neighborhood Longitude']]
df_tmp_2 = df_tmp.drop_duplicates()
df_tmp_2.shape

(38, 3)

In [68]:
toronto_merged = toronto_grouped_clustering

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_
toronto_merged['Neighborhood'] = toronto_grouped[['Neighborhood']]

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged = toronto_merged.join(df_tmp_2.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Venue Category_Accessories Store,Venue Category_Adult Boutique,Venue Category_Afghan Restaurant,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,Venue Category_American Restaurant,...,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Neighborhood Latitude,Neighborhood Longitude
0,0.01,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,...,Venue Category_Steakhouse,Venue Category_American Restaurant,Venue Category_Thai Restaurant,Venue Category_Gym,Venue Category_Restaurant,Venue Category_Hotel,Venue Category_Clothing Store,Venue Category_Bar,43.650571,-79.384568
1,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,Venue Category_Restaurant,Venue Category_Farmers Market,Venue Category_Steakhouse,Venue Category_Bakery,Venue Category_Cheese Shop,Venue Category_Café,Venue Category_Seafood Restaurant,Venue Category_Pub,43.644771,-79.373306
2,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,Venue Category_Café,Venue Category_Breakfast Spot,Venue Category_Yoga Studio,Venue Category_Stadium,Venue Category_Grocery Store,Venue Category_Furniture / Home Store,Venue Category_Italian Restaurant,Venue Category_Falafel Restaurant,43.636847,-79.428191
3,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,Venue Category_Garden Center,Venue Category_Restaurant,Venue Category_Recording Studio,Venue Category_Auto Workshop,Venue Category_Skate Park,Venue Category_Brewery,Venue Category_Fast Food Restaurant,Venue Category_Spa,43.662744,-79.321558
4,0.00,0.0,0.0,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,0.00,...,Venue Category_Airport Terminal,Venue Category_Boutique,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Harbor / Marina,Venue Category_Boat or Ferry,Venue Category_Sculpture Garden,43.628947,-79.394420


In [69]:
toronto_merged.shape

(38, 252)

In [70]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Neighborhood Latitude'], toronto_merged['Neighborhood Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters